In [2]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('reports_v5.db')

# Read SQL query into a DataFrame
query = "SELECT * FROM reports"
df = pd.read_sql(query, conn, index_col='report_id')

# Define a function to calculate total debt
def calculate_total_debt(row):
    if row['DebtNoncurrent'] != 0:
        return row['DebtNoncurrent'] + row['DebtCurrent']
    elif row['LongTermDebtNoncurrent'] != 0:
        return row['LongTermDebtNoncurrent'] + row['DebtCurrent']
    else:
        return row['CapitalLeaseObligationsNoncurrent'] + row['FinanceLeaseLiabilityNoncurrent'] + \
               row['CapitalLeaseObligationsCurrent'] + row['LinesOfCreditCurrent'] + row['FinanceLeaseLiabilityCurrent']

# Apply the function to create a new column 'total_debt'
df['total_debt'] = df.apply(calculate_total_debt, axis=1)

# Print the DataFrame with the new column
print(df['total_debt'])


0        2.872000e+06
1        0.000000e+00
2        0.000000e+00
3        0.000000e+00
4        1.375088e+09
             ...     
29367    0.000000e+00
29368    0.000000e+00
29369    4.400000e+06
29370    0.000000e+00
29371    0.000000e+00
Name: total_debt, Length: 29372, dtype: float64


In [3]:
# Calculate the total number of entries
total_entries = len(df)

# Calculate the number of entries where total_debt is equal to 0
num_zero_debt_entries = (df['total_debt'] == 0).sum()

# Calculate the percentage
percent_zero_debt_entries = (num_zero_debt_entries / total_entries) * 100

# Print the percentage
print("Percentage of entries with total_debt equal to 0:", percent_zero_debt_entries, "%")

Percentage of entries with total_debt equal to 0: 70.19950973716465 %


In [5]:
import pandas as pd
import json

# Read the JSON file into a Python dictionary
with open('report_data_v2.json', 'r') as f:
    report_data = json.load(f)

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['key', 'debt_to_asset_ratio_median', 'debt_to_book_equity_median'])

# Iterate over each key in the report data
for key, reports in report_data.items():
    # Create a list to store debt to asset ratios and debt to book equity ratios for each report_id
    debt_to_asset_ratios = []
    debt_to_book_equity_ratios = []
    
    # Iterate over each report in the current key
    for report in reports:
        report_id = report['report_id']
        
        # Calculate debt to asset ratio and debt to book equity ratio
        total_debt = df.loc[report_id, 'total_debt']
        assets = df.loc[report_id, 'Assets']
        stockholders_equity = df.loc[report_id, 'StockholdersEquity']
        
        debt_to_asset_ratio = total_debt / assets
        debt_to_book_equity_ratio = total_debt / stockholders_equity
        
        # Append the ratios to the lists
        debt_to_asset_ratios.append(debt_to_asset_ratio)
        debt_to_book_equity_ratios.append(debt_to_book_equity_ratio)
    
    # Calculate the median of debt to asset ratios and debt to book equity ratios
    median_debt_to_asset_ratio = pd.Series(debt_to_asset_ratios).median()
    median_debt_to_book_equity_ratio = pd.Series(debt_to_book_equity_ratios).median()
    
    # Append the results to the results DataFrame
    results_df = results_df.append({'key': key,
                                    'debt_to_asset_ratio_median': median_debt_to_asset_ratio,
                                    'debt_to_book_equity_median': median_debt_to_book_equity_ratio},
                                   ignore_index=True)

# Print the results
print(results_df)


AttributeError: 'DataFrame' object has no attribute 'append'